### まずはkこの数字でvalを作れるかを判定する関数を作成

In [13]:
from itertools import permutations, product

def can_achieve_value(arr, val):
    operators = ['+', '-', '*', '/']
    n = len(arr)

    # 配列の要素の全ての可能な順列を生成する
    all_permutations = permutations(arr)

    for permutation in all_permutations:
        # それぞれの順列に対して、全ての可能な演算子の組み合わせを試す
        all_operator_combinations = product(operators, repeat=n-1)
        for operator_combination in all_operator_combinations:
            expression = f"{permutation[0]}"
            for i in range(1, n):
                expression += f" {operator_combination[i-1]} {permutation[i]}"
            # 式を評価し、目標の値と一致するかどうかを確認する
            try:
                if eval(expression) == val:
                    return True, expression  # 評価が目標の値と一致する場合、Trueと式を返す
            except ZeroDivisionError:
                continue  # 0で除算するエラーをキャッチし、無視する

    return False, ""  # どの組み合わせも目標の値を達成できない場合、Falseと空の文字列を返す

# 呼び出し例
result, expression = can_achieve_value([3, 8, 3, 8], 24)
print(result, expression)  # 出力: True "1 + 2 + 3 + 4"


False 


上記判定法だと[3,8,3,8]->24(#8 /(3-8/3)=24)が探し出せない
以下改良案(逆ポーランド記法を用いて表現力と一位性を構成)

In [49]:
from itertools import permutations, product
import math

def eval_rpn(expression):
    stack = []
    for token in expression:
        if token in {'+', '-', '*', '/'}:
            # スタックに少なくとも2つの要素があることを確認する
            if len(stack) < 2:
                return None
            b = stack.pop()
            a = stack.pop()
            if token == '+':
                stack.append(a + b)
            elif token == '-':
                stack.append(a - b)
            elif token == '*':
                stack.append(a * b)
            elif token == '/':
                # 0での除算を避ける
                if b == 0:
                    return None
                stack.append(a / b)
        else:
            stack.append(float(token))  # 数字を浮動小数点数として扱う
    # スタックに1つの要素が残っていることを確認する
    return stack[0] if len(stack) == 1 else None

def solve_rpn(nums, target):
    # 数字と演算子の順列を生成
    nums = list(map(lambda num: str(num), nums))
    op_perms = product(['+', '-', '*', '/'], repeat=len(nums)-1)
    
    for op_perm in op_perms:
        for perm in permutations(list(op_perm) + nums):
            # RPN式を評価
            result = eval_rpn(perm)
            if result is not None and abs(result - target) < 1e-8:
                return perm  # RPN式を返す

    return None  # 解が見つからない場合はNoneを返す

# 例の呼び出し
nums = [3, 8, 3, 8]
target = 24
solution = solve_rpn(nums, target)
if solution:
    print(' '.join(map(str, solution)))  # RPN式を出力する
else:
    print("No solution found")

8 3 8 3 / - /


### make10を1~9でできないものがあるかを判定

In [57]:
for i in range(1,10):
    for j in range(i+1,10):
        for k in range(j+1,10):
            for l in range(k+1,10):
                nums = [i,j,k,l]
                solution = solve_rpn(nums, target)
                if not solution:
                    print(nums, "cannot be solved")


[1, 6, 7, 8] cannot be solved
[3, 4, 6, 7] cannot be solved
